In [198]:
import os
import re
import pathlib

import nltk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from wordcloud import WordCloud
from PIL import Image
from ydata_profiling import ProfileReport

In [199]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pyramidheadshark\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [200]:
morph = MorphAnalyzer()

In [201]:
DF_PATH = pathlib.Path(os.getcwd()) / ".." / "data" / "processed" / "dataset.csv"

In [202]:
df = pd.read_csv(
    DF_PATH    
)

df_copy = df.copy()

categorial_dfs = {
    str(category): \
        df[df['Category'] == category].reset_index(drop=True) \
        for category in df['Category'].unique()
}

In [203]:
def clean_text_from_specsymbols(text):
    return re.sub(r"[^a-zA-Zа-яА-ЯёЁ\s]", "", text)

def clean_text_from_single_letters(text):
    return re.sub(r"\b[а-яёa-zA-Z0-9]\b", "", text, flags=re.IGNORECASE)

def clean_text_from_double_letters(text):
    return re.sub(r"\b[a-zA-Zа-яА-Я]{2}\b", "", text, flags=re.IGNORECASE)

def clean_text_from_unneccesary(text):
    words = nltk.word_tokenize(text)
    
    russian_stopwords = stopwords.words('russian')

    clean_words = []
    for word in words:
        lemma = morph.parse(word)[0].normal_form
        if lemma not in russian_stopwords:
            clean_words.append(lemma)

    return ' '.join(clean_words)

In [204]:
for category, df in categorial_dfs.items():
    df['Description'] = df['Description'].apply(clean_text_from_specsymbols)
    df['Description'] = df['Description'].apply(clean_text_from_single_letters)
    df['Description'] = df['Description'].apply(clean_text_from_double_letters)
    df['Description'] = df['Description'].apply(clean_text_from_unneccesary)


In [205]:
def get_word_counts(df):
    word_freq = {}
    for text in df['Description']:
        words = text.split()
        for word in words:
            lemma = morph.parse(word)[0].normal_form
            if lemma not in stopwords.words('russian'):
                word_freq[lemma] = word_freq.get(lemma, 0) + 1
    return word_freq
                
def get_wordcloud(word_freq):
    wordcloud = WordCloud(
        max_words=50,
        width=1200,
        height=1000,
        background_color='white',
        colormap='Blues_r'
        ).generate_from_frequencies(word_freq)
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    
def get_wordcloud_slonik(word_freq):
    mask = np.array(Image.open('logo.png'))
    
    wordcloud = WordCloud(
        scale=4,
        width=1200,
        height=1200,
        mask=mask,
        background_color='white',
        colormap='Blues_r',
        contour_color='#ADD8E6',
        contour_width=1
        ).generate_from_frequencies(word_freq)
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.savefig('test.png')
    plt.show()

In [206]:
def create_report(df, name):
    profile = ProfileReport(df, title="Summarize dataset")
    profile.to_file(f"report_{name}.html")

---

И ещё желательно в переменные среды передать word_freq на каждое множество, чтобы фича могла взаимодействовать адекватно

In [210]:
for key, value in categorial_dfs.items():
    print(value)
    df = value
    create_report(df, key)

                                            Description Category
0                                    турпоездка договор  SERVICE
1                               оказание услуга договор  SERVICE
2                                       лечение договор  SERVICE
3                                       лечение договор  SERVICE
4                                оказание прочий услуга  SERVICE
...                                                 ...      ...
2382                   медицинский обслуживание договор  SERVICE
2383                       оказание услуга договор июнь  SERVICE
2384  транспортировка перевозка автомобиль коломна э...  SERVICE
2385  оплата сентябрь транспортный услуга ндс облага...  SERVICE
2386                участие конференция саранск договор  SERVICE

[2387 rows x 2 columns]


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

                                           Description     Category
0    предвариельный договор октябрь квартира адрес ...  REAL_ESTATE
1    предвариельный договор квартира адрес тольятти...  REAL_ESTATE
2              обеспечительный платёж виллудоговор май  REAL_ESTATE
3    предвариельный договор июнь квартира адрес пуш...  REAL_ESTATE
4    оплата догвор куплипродажа вилла адрес северск...  REAL_ESTATE
..                                                 ...          ...
878  приобретение недвижимость адрес абакан дорожны...  REAL_ESTATE
879     оплата счёт аренда помещение договор субаренда  REAL_ESTATE
880   доплата счёт май аренда помещение январь договор  REAL_ESTATE
881  паевой взнос жилищностроительный кооператив це...  REAL_ESTATE
882  предвариельный договор квартира адрес бердск л...  REAL_ESTATE

[883 rows x 2 columns]


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]